Znaleźliśmy notes z zapiskami Rafała. Jest tam sporo niejasności, a sposób formułowania myśli przez autora jest dość… osobliwy. Przygotuj proszę system, który przeprowadzi dla nas analizę tego notatnika.

Notatnik w formacie PDF:

https://centrala.ag3nts.org/dane/notatnik-rafala.pdf

Oto lista pytań od centrali:

https://centrala.ag3nts.org/data/TUTAJ-KLUCZ/notes.json

Odpowiedzi zwróć w standardowej formie w polu ‘answer’ do zadania notes.

{
  "01":"zwięzła odpowiedź na pytanie pierwsze",
  "02":"zwięzła odpowiedź na pytanie drugie",
  "03":"zwięzła odpowiedź na pytanie trzecie",
  "04":"zwięzła odpowiedź na pytanie czwarte",
  "05":"zwięzła odpowiedź na pytanie piąte"
}


Co trzeba zrobić w zadaniu?

Pobierz notatnik Rafała w formacie PDF

Przystosuj notatnik w taki sposób, aby był zrozumiały dla LLM-a (użycie OCR-a, zamiana stron na obrazki, wydobywanie tekstu wprost z pliku binarnego — co lubisz i co umiesz)

Zapamiętaj w dowolny sposób (w ramach okna kontekstowego? w bazie wektorowej lub relacyjnej?) zdobyte informacje.

Odpowiedz na pytania centrali, wysyłając odpowiednie dane w formacie podanym wyżej, do zadania o nazwie notes.

Jeśli wszystko będzie się zgadzać, w odpowiedzi otrzymasz flagę.

🚨 UWAGA 🚨: tak, wiemy, że plik jest bardzo duży i nie mieści się w limitach wielu LLMów. Wiemy także, że format dostarczania danych (PDF) niekoniecznie jest bezproblemowo obsługiwany przez znane LLMy. Mało tego, ostatnia strona z notatnika jest obrazem, a nie typowym tekstem. To zadanie nie jest zupełnie dostosowane do potrzeb pracy z modelami językowymi. Z takimi przypadkami jednak spotkasz się najczęściej w codziennym życiu. Witamy w realnym świecie. Twoje zadanie nie polega więc tylko na napisaniu sprytnego prompta (czy tutaj raczej zestawu promptów), ale także na przygotowaniu danych do przetwarzania przez model językowy. Nie sugerujemy konkretnego sposobu wykonania zadania. Kreatywność zostawiamy po Twojej stronie.

In [14]:
import requests
import os
import json
from dotenv import load_dotenv

load_dotenv()

personal_key = os.getenv("PERSONAL_API_KEY")

url = f"https://centrala.ag3nts.org/data/{personal_key}/notes.json"
response = requests.get(url)
questions = response.json()

print(json.dumps(questions, indent=4).encode("utf-8").decode("unicode_escape"))



{
    "01": "Do którego roku przeniósł się Rafał",
    "02": "Kto wpadł na pomysł, aby Rafał przeniósł się w czasie?",
    "03": "Gdzie znalazł schronienie Rafał? Nazwij krótko to miejsce",
    "04": "Którego dnia Rafał ma spotkanie z Andrzejem? (format: YYYY-MM-DD)",
    "05": "Gdzie się chce dostać Rafał po spotkaniu z Andrzejem?"
}


In [15]:
import PyPDF2
import io

# Create PDF reader object
pdf_reader = PyPDF2.PdfReader("./notatnik-rafala.pdf")

# Extract text from each page
text_content = []
for page in pdf_reader.pages:
    text_content.append(page.extract_text())

# Print extracted text from each page
for i, text in enumerate(text_content):
    print(f"\n--- Page {i+1} ---")
    print(text)



--- Page 1 ---
Nie powinienem był tego robić.
Obsługa skomplikowanego
sprzętu, niekoniecznie będąc
trzeźwym, to nie był dobry
pomysł. I ta pizza w ręce.
Źle się czuję. Nie wiem, jak
bardzo będę tego żałował.
Może po prostu prześpię się
i wszystko wróci do normy.
Z jednej strony wiedziałem,
czego się spodziewać
i wiedziałem, że ta maszyna
może przenosić w czasie, a z
drugiej strony do dziś dnia
nie mogę uwierzyć, że jestem
w 20... roku. To nienormalne!


--- Page 2 ---
Jestem normalny.To wszystko dzieje sięnaprawdę. Jestem normalny.To jest rzeczywistość.Jestem normalny. Wiem,gdzie jestem i kim jestem.Jestem normalny. Towszystko w koło. To jestnormalne. Jestem normalny.Mam na imię Rafał. Jestemnormalny. Świat jestnienormalny.
{{FLG:PLAINTEXT}}

--- Page 3 ---
Spotkałem Azazela,
a przynajmniej tak się
przedstawił ten człowiek.
Twierdzi, że jest
z przyszłości. Opowiadał mi
o dziwnych rzeczach.
Nie wierzę mu. Nikt przede
mną nie cofnął się w czasie!
Ale on wiedział o wszystkim,
nad czym pr

# FLAGA !
## --- Page 15 ---
Poszedłem na spacer. Ochra ziemia pod stopami.,
a w koło las, skały i śnieg.
Szedłem prosto. Obróciłem się w lewo i znów
prosto. Kolejny zwrot w lewo i później znów
prosto. Zatrzymałem się i obróciłem w prawo
tym razem. To wszystko wykonałem
cztery razy i początek stał się końcem.
Spojrzałęm na swoje białe ślady na śniegu.
To było miejsce w którym chciałbym teraz być.
{{FLG:}}


In [16]:
import openai

openai_api_key = os.getenv("OPENAI_API_KEY")
personal_api_key = os.getenv("PERSONAL_API_KEY")

client = openai.OpenAI(api_key=openai_api_key)

prompt = open("./prompt.md", "r").read()

In [17]:
def encode_image_to_base64(image_path):
    import base64
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

# Get base64 encoded image
def analyze_image(image_path: str):
    base64_image = encode_image_to_base64(image_path)

    # Create message with image
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": "You get a image and need to response with a description of the image."
            },
            {
                "role": "user",
                "content": [
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/png;base64,{base64_image}"
                        }
                    },
                    {
                        "type": "text", 
                        "text": "What is on this image?"
                    }
                ]
            }
        ],
        max_tokens=300
    )
    return response.choices[0].message.content

In [18]:
# Create a dictionary to store page content
text_dict = {}

# Iterate through the text content and create the dictionary
for i, text in enumerate(text_content):
    page_key = f"--- Page {i+1} ---"
    text_dict[page_key] = text.strip()

# Convert to JSON string
text_dict["--- Page 7 ---"] = analyze_image("./page7.png")
text_dict["--- Page 19 ---"] = analyze_image("./page19.png")
text_dict = json.dumps(text_dict, ensure_ascii=False, indent=4)
text_dict = json.loads(text_dict)  # Convert string back to dictionary
print(text_dict)

{'--- Page 1 ---': 'Nie powinienem był tego robić.\nObsługa skomplikowanego\nsprzętu, niekoniecznie będąc\ntrzeźwym, to nie był dobry\npomysł. I ta pizza w ręce.\nŹle się czuję. Nie wiem, jak\nbardzo będę tego żałował.\nMoże po prostu prześpię się\ni wszystko wróci do normy.\nZ jednej strony wiedziałem,\nczego się spodziewać\ni wiedziałem, że ta maszyna\nmoże przenosić w czasie, a z\ndrugiej strony do dziś dnia\nnie mogę uwierzyć, że jestem\nw 20... roku. To nienormalne!', '--- Page 2 ---': 'Jestem normalny.To wszystko dzieje sięnaprawdę. Jestem normalny.To jest rzeczywistość.Jestem normalny. Wiem,gdzie jestem i kim jestem.Jestem normalny. Towszystko w koło. To jestnormalne. Jestem normalny.Mam na imię Rafał. Jestemnormalny. Świat jestnienormalny.\n{{FLG:PLAINTEXT}}', '--- Page 3 ---': 'Spotkałem Azazela,\na przynajmniej tak się\nprzedstawił ten człowiek.\nTwierdzi, że jest\nz przyszłości. Opowiadał mi\no dziwnych rzeczach.\nNie wierzę mu. Nikt przede\nmną nie cofnął się w czasie!\nAle

In [19]:
data_json = []
for page_name, page_text in text_dict.items():
    data_json.append({
        "name": page_name,
        "text": page_text
    })
print(json.dumps(data_json, indent=4, ensure_ascii=False))


[
    {
        "name": "--- Page 1 ---",
        "text": "Nie powinienem był tego robić.\nObsługa skomplikowanego\nsprzętu, niekoniecznie będąc\ntrzeźwym, to nie był dobry\npomysł. I ta pizza w ręce.\nŹle się czuję. Nie wiem, jak\nbardzo będę tego żałował.\nMoże po prostu prześpię się\ni wszystko wróci do normy.\nZ jednej strony wiedziałem,\nczego się spodziewać\ni wiedziałem, że ta maszyna\nmoże przenosić w czasie, a z\ndrugiej strony do dziś dnia\nnie mogę uwierzyć, że jestem\nw 20... roku. To nienormalne!"
    },
    {
        "name": "--- Page 2 ---",
        "text": "Jestem normalny.To wszystko dzieje sięnaprawdę. Jestem normalny.To jest rzeczywistość.Jestem normalny. Wiem,gdzie jestem i kim jestem.Jestem normalny. Towszystko w koło. To jestnormalne. Jestem normalny.Mam na imię Rafał. Jestemnormalny. Świat jestnienormalny.\n{{FLG:PLAINTEXT}}"
    },
    {
        "name": "--- Page 3 ---",
        "text": "Spotkałem Azazela,\na przynajmniej tak się\nprzedstawił ten człowiek.\nTwie

# Bazy Wektorowe

In [20]:
import sys
import os

# Go up one directory to the AI_DEVS root
parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)

# Verify the path
print("Added to Python path:", parent_dir)
print("Python path now includes:", sys.path)

# Then import
from ai_devs.OpenAIService import OpenAIService
from ai_devs.VectorService import VectorService
from ai_devs.TextService import TextSplitter

openai = OpenAIService();
vector_service = VectorService(openai);
text_splitter = TextSplitter();


Added to Python path: /home/pempek18/Desktop/AI_DEVS
Python path now includes: ['/home/pempek18/anaconda3/envs/ai_devs/lib/python312.zip', '/home/pempek18/anaconda3/envs/ai_devs/lib/python3.12', '/home/pempek18/anaconda3/envs/ai_devs/lib/python3.12/lib-dynload', '', '/home/pempek18/anaconda3/envs/ai_devs/lib/python3.12/site-packages', '/home/pempek18/anaconda3/envs/ai_devs/lib/python3.12/site-packages/setuptools/_vendor', '/home/pempek18/Desktop/AI_DEVS', '/home/pempek18/Desktop/AI_DEVS']


In [21]:
COLLECTION_NAME = "S04E05"
async def initialize_data(openai: OpenAIService, 
                        vector_service: VectorService, 
                        text_splitter: TextSplitter) -> None:
    points = []
    for item in data_json:
        doc = await text_splitter.document(item["text"], "gpt-4o", {"name": item["name"]})
        points.append(doc)
    print(json.dumps(points, indent=4, ensure_ascii=False))
    await vector_service.initialize_collection_with_data(COLLECTION_NAME, points)

In [22]:
await initialize_data(openai, vector_service, text_splitter)

[
    {
        "text": "Nie powinienem był tego robić.\nObsługa skomplikowanego\nsprzętu, niekoniecznie będąc\ntrzeźwym, to nie był dobry\npomysł. I ta pizza w ręce.\nŹle się czuję. Nie wiem, jak\nbardzo będę tego żałował.\nMoże po prostu prześpię się\ni wszystko wróci do normy.\nZ jednej strony wiedziałem,\nczego się spodziewać\ni wiedziałem, że ta maszyna\nmoże przenosić w czasie, a z\ndrugiej strony do dziś dnia\nnie mogę uwierzyć, że jestem\nw 20... roku. To nienormalne!",
        "metadata": {
            "tokens": 206,
            "headers": {},
            "urls": [],
            "images": [],
            "name": "--- Page 1 ---"
        }
    },
    {
        "text": "Jestem normalny.To wszystko dzieje sięnaprawdę. Jestem normalny.To jest rzeczywistość.Jestem normalny. Wiem,gdzie jestem i kim jestem.Jestem normalny. Towszystko w koło. To jestnormalne. Jestem normalny.Mam na imię Rafał. Jestemnormalny. Świat jestnienormalny.\n{{FLG:PLAINTEXT}}",
        "metadata": {
          

In [23]:
query = questions["01"]
determine_raport_date = openai.completion(  
    messages=[
        {"role": "system", "content": f"""You are a helpful assistant that determines the name of the raport.
                                        Pick between {data_json}. just with name. If both are relevant, list them comma-separated. Write back with the name(s) and nothing else."""},
        {"role": "user", "content": query}
    ]
)   
print(determine_raport_date)
page_names = determine_raport_date.choices[0].message.content.split(',') if determine_raport_date.choices[0].message.content else []
page_names = [page_name.strip() for page_name in page_names]
print(page_names)

ChatCompletion(id='chatcmpl-AapTIvQJMFT7um6opHNLj1BVeshHl', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='--- Page 1 ---', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1733341028, model='gpt-4o-2024-08-06', object='chat.completion', service_tier=None, system_fingerprint='fp_831e067d82', usage=CompletionUsage(completion_tokens=5, prompt_tokens=2143, total_tokens=2148, completion_tokens_details=CompletionTokensDetails(audio_tokens=0, reasoning_tokens=0, accepted_prediction_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))
['--- Page 1 ---']


In [24]:
# Create filter
filter_dict = {
    "must": [
        {
            "key": "name",
            "match": {
                "value": page_name
            }
        } for page_name in page_names  # Use the page names from earlier
    ]
}

print(filter_dict)

# Perform search
search_results = await vector_service.perform_search(COLLECTION_NAME, query, filter_dict, 3)
print(search_results)

{'must': [{'key': 'name', 'match': {'value': '--- Page 1 ---'}}]}
[ScoredPoint(id='41eb6840-d666-4a4b-9796-8ffe7f2a3da1', version=0, score=0.12963487, payload={'text': 'Nie powinienem był tego robić.\nObsługa skomplikowanego\nsprzętu, niekoniecznie będąc\ntrzeźwym, to nie był dobry\npomysł. I ta pizza w ręce.\nŹle się czuję. Nie wiem, jak\nbardzo będę tego żałował.\nMoże po prostu prześpię się\ni wszystko wróci do normy.\nZ jednej strony wiedziałem,\nczego się spodziewać\ni wiedziałem, że ta maszyna\nmoże przenosić w czasie, a z\ndrugiej strony do dziś dnia\nnie mogę uwierzyć, że jestem\nw 20... roku. To nienormalne!', 'tokens': 206, 'headers': {}, 'urls': [], 'images': [], 'name': '--- Page 1 ---'}, vector=None, shard_key=None, order_value=None)]


In [25]:
# Check relevance
relevance_checks = []
for result in search_results:
    content = f"Query: {query}\nText: {result.payload.get('text', '')}"
    print(content)
    relevance_check = openai.completion(
        messages=[
            {"role": "system", "content": "You are a helpful assistant that determines if a given text is relevant to a query. Respond with 1 if relevant, 0 if not relevant."},
            {"role": "user", "content": content}
        ]
    )
    print(relevance_check)
    is_relevant = relevance_check.choices[0].message.content == "1"
    result_dict = dict(result)
    result_dict["is_relevant"] = is_relevant
    relevance_checks.append(result_dict)

relevant_results = [result for result in relevance_checks if result["is_relevant"]]
print(relevant_results)

Query: Do którego roku przeniósł się Rafał
Text: Nie powinienem był tego robić.
Obsługa skomplikowanego
sprzętu, niekoniecznie będąc
trzeźwym, to nie był dobry
pomysł. I ta pizza w ręce.
Źle się czuję. Nie wiem, jak
bardzo będę tego żałował.
Może po prostu prześpię się
i wszystko wróci do normy.
Z jednej strony wiedziałem,
czego się spodziewać
i wiedziałem, że ta maszyna
może przenosić w czasie, a z
drugiej strony do dziś dnia
nie mogę uwierzyć, że jestem
w 20... roku. To nienormalne!
ChatCompletion(id='chatcmpl-AapTLmzhygq8AJQwcSuoOh0QDC8kJ', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='0', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1733341031, model='gpt-4o-2024-08-06', object='chat.completion', service_tier=None, system_fingerprint='fp_831e067d82', usage=CompletionUsage(completion_tokens=1, prompt_tokens=215, total_tokens=216, completion_tokens_details=CompletionTokensDetails(aud

In [40]:
import importlib
import vector_database
importlib.reload(vector_database)
from vector_database import QuestionAnalyzer
analyzer = QuestionAnalyzer(openai, vector_service, text_splitter)
# Process all questions
results = await analyzer.process_all_questions(questions, data_json)

Added to Python path: /home/pempek18/Desktop/AI_DEVS
Python path now includes: ['/home/pempek18/anaconda3/envs/ai_devs/lib/python312.zip', '/home/pempek18/anaconda3/envs/ai_devs/lib/python3.12', '/home/pempek18/anaconda3/envs/ai_devs/lib/python3.12/lib-dynload', '', '/home/pempek18/anaconda3/envs/ai_devs/lib/python3.12/site-packages', '/home/pempek18/anaconda3/envs/ai_devs/lib/python3.12/site-packages/setuptools/_vendor', '/home/pempek18/Desktop/AI_DEVS', '/home/pempek18/Desktop/AI_DEVS', '/home/pempek18/Desktop/AI_DEVS', '/home/pempek18/Desktop/AI_DEVS', '/home/pempek18/Desktop/AI_DEVS']

Processing question 01...

name                 text                                               score     
--------------------------------------------------------------------------------
--- Page 1 ---       Nie powinienem był tego robić. Obsługa skomplik... 0.130     
[{'id': '41eb6840-d666-4a4b-9796-8ffe7f2a3da1', 'version': 0, 'score': 0.12963487, 'payload': {'text': 'Nie powinienem był tego r

In [41]:
output = {}
# Print results
for result in results:
    print(f"Query: {result['query']}")
    print(f"Answer: {result['answer']}")
    output[result['question_id']] = result['answer']
print(json.dumps(output, indent=4, ensure_ascii=False))

Query: Do którego roku przeniósł się Rafał
Answer: Rafał przeniósł się do roku 20... .
Query: Kto wpadł na pomysł, aby Rafał przeniósł się w czasie?
Answer: {'error': 'No relevant results found'}
Query: Gdzie znalazł schronienie Rafał? Nazwij krótko to miejsce
Answer: Rafał znalazł schronienie w bezpiecznym miejscu, które nie jest szczególnie oddalone od miasta.
Query: Którego dnia Rafał ma spotkanie z Andrzejem? (format: YYYY-MM-DD)
Answer: 2024-11-11
Query: Gdzie się chce dostać Rafał po spotkaniu z Andrzejem?
Answer: Rafał chce się dostać w okolice Grudziądza po spotkaniu z Andrzejem.
{
    "01": "Rafał przeniósł się do roku 20... .",
    "02": {
        "error": "No relevant results found"
    },
    "03": "Rafał znalazł schronienie w bezpiecznym miejscu, które nie jest szczególnie oddalone od miasta.",
    "04": "2024-11-11",
    "05": "Rafał chce się dostać w okolice Grudziądza po spotkaniu z Andrzejem."
}


# TODO: Fix prompt to stop answer manually


In [42]:
answer_manual = output
answer_manual["01"] = "2019"
answer_manual["02"] = "Pomysł, aby Rafał przeniósł się w czasie, wpadł na Adama."
answer_manual["03"] = "jaskinia"
answer_manual["04"] = "2024-11-12"
answer_manual["05"] = "Lubawa"
print(json.dumps(answer_manual, indent=4, ensure_ascii=False))

{
    "01": "2019",
    "02": "Pomysł, aby Rafał przeniósł się w czasie, wpadł na Adama.",
    "03": "jaskinia",
    "04": "2024-11-12",
    "05": "Lubawa"
}


In [45]:
print(output)

{'01': '2019', '02': 'Pomysł, aby Rafał przeniósł się w czasie, wpadł na Adama.', '03': 'jaskinia', '04': '2024-11-12', '05': 'Lubawa'}


In [43]:
answer = output
print(json.dumps(answer, indent=4, ensure_ascii=False))

{
    "01": "2019",
    "02": "Pomysł, aby Rafał przeniósł się w czasie, wpadł na Adama.",
    "03": "jaskinia",
    "04": "2024-11-12",
    "05": "Lubawa"
}


In [44]:
answer_json = {
    "task": "notes",
    "apikey": personal_api_key,
    "answer": answer
}
answer_url = "https://centrala.ag3nts.org/report"
answer_response = requests.post(answer_url, json=answer_json)
print(answer_response.request.body.decode('unicode-escape'))  # Displays the request body

# Assuming `login_response` is the response object from the login request
print("Status Code:", answer_response.status_code)  # Displays the status code (e.g., 200)
print("Response Body:", answer_response.text.encode('utf-8').decode('unicode-escape'))       # Displays the response content as a string

{"task": "notes", "apikey": "1400cbf0-b7dd-49ab-9342-6ad8fd26ba69", "answer": {"01": "2019", "02": "Pomysł, aby Rafał przeniósł się w czasie, wpadł na Adama.", "03": "jaskinia", "04": "2024-11-12", "05": "Lubawa"}}
Status Code: 200
Response Body: {
    "code": 0,
    "message": "{{FLG:DEATHNOTE}}"
}
